In [11]:
import os
import json
import utils

import numpy as np
import pandas as pd

DATA_PATH = '../../data/real/availability/2023-02-15/parsed'
# DATA_PATH = '../../data/real/ping_based'
gap_seconds = 0

df = pd.DataFrame()
records = []

for env_name in os.listdir(DATA_PATH):
    if not env_name.endswith('.json'):
        continue

    f = json.load(open(os.path.join(DATA_PATH, env_name)))
    gap_seconds = f['metadata']['gap_seconds']
    preemption_trace = np.array(f['data'])

    wait_times = utils.get_consecutive_1s(preemption_trace) * gap_seconds / 3600
    lifetime = utils.get_consecutive_1s(1 - preemption_trace) * gap_seconds / 3600

    records.append({
        'env': env_name,
        'avg_lifetime': lifetime.mean(),
        'std_lifetime': lifetime.std(),
        'max_lifetime': lifetime.max(),
        'min_lifetime': lifetime.min(),
        'p25_lifetime': np.percentile(lifetime, 25),
        'p75_lifetime': np.percentile(lifetime, 75),
        'avg_wait_time': wait_times.mean(),
        'std_wait_time': wait_times.std(),
        'max_wait_time': wait_times.max(),
        'min_wait_time': wait_times.min(),
        'p25_wait_time': np.percentile(wait_times, 25),
        'p75_wait_time': np.percentile(wait_times, 75),
        # 'wait_time': wait_times,
        # 'lifetime': lifetime,
        })

df = pd.DataFrame.from_records(records)
df.to_csv(os.path.join(DATA_PATH, 'stats.tsv'), index=False, sep='\t')
df

,env,avg_lifetime,std_lifetime,max_lifetime,min_lifetime,p25_lifetime,p75_lifetime,avg_wait_time,std_wait_time,max_wait_time,min_wait_time,p25_wait_time,p75_wait_time
0,us-west-2a_v100_1.json,6.064282,15.510487,126.100000,0.1625,0.325000,4.550000,0.808043,1.145568,8.070833,0.1625,0.325000,0.704167
1,us-west-2b_v100_1.json,10.401140,25.869370,135.741667,0.1625,0.216667,6.147917,1.104078,1.484259,7.204167,0.1625,0.325000,1.151042
2,us-east-1d_v100_1.json,1.661111,3.549106,41.762500,0.1625,0.216667,1.245833,2.052806,4.816928,56.225000,0.1625,0.243750,1.950000
3,us-east-2a_v100_1.json,5.678596,13.657108,115.645833,0.1625,0.325000,4.170833,1.800114,3.111619,20.854167,0.1625,0.216667,1.868750
4,us-east-1f_v100_1.json,2.257008,4.572275,34.937500,0.1625,0.216667,1.895833,1.560795,2.859916,23.887500,0.1625,0.162500,1.327083
5,us-west-2c_v100_1.json,6.805682,17.828438,143.379167,0.1625,0.325000,4.089583,0.835769,1.243782,8.450000,0.1625,0.216667,0.744792
6,us-east-1c_v100_1.json,1.480766,3.146366,30.170833,0.1625,0.216667,1.272917,1.693338,2.792383,23.345833,0.1625,0.216667,1.909375
7,us-east-1a_v100_1.json,0.719368,1.072819,8.070833,0.1625,0.162500,0.704167,3.582251,9.221246,80.220833,0.1625,0.176042,2.491667
8,us-east-2b_v100_1.json,4.256571,7.516235,49.183333,0.1625,0.352083,3.900000,1.982810,2.893557,20.800000,0.1625,0.325000,2.897917
